<a href="https://colab.research.google.com/github/hrnnikolov/ML_models/blob/main/model_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Getting Data



In [2]:
# # For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
# try:
#     import torch
#     import torchvision
#     assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
#     assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
#     print(f"torch version: {torch.__version__}")
#     print(f"torchvision version: {torchvision.__version__}")
# except:
#     print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
#     !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#     import torch
#     import torchvision
#     print(f"torch version: {torch.__version__}")
#     print(f"torchvision version: {torchvision.__version__}")

In [3]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4056, done.
remote: Total 4056 (delta 0), reused 0 (delta 0), pack-reused 4056
Receiving objects: 100% (4056/4056), 646.90 MiB | 33.17 MiB/s, done.
Resolving deltas: 100% (2371/2371), done.
Updating files: 100% (248/248), done.


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
# Download pizza, steak, sushi images from GitHub
data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

data_20_percent_path

[INFO] Did not find data/pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip data...


PosixPath('data/pizza_steak_sushi_20_percent')

In [6]:
train_dir = data_20_percent_path / 'train'
test_dir = data_20_percent_path / 'test'

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi_20_percent/train'),
 PosixPath('data/pizza_steak_sushi_20_percent/test'))

#Creating Effnetb2 model

In [7]:
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT

effnetb2_transforms = effnetb2_weights.transforms()

effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

for param in effnetb2.parameters():
  param.requires_grad = False

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 87.6MB/s]


In [8]:
effnetb2.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1408, out_features=1000, bias=True)
)

In [9]:
effnetb2.classifier = nn.Sequential(nn.Dropout(p=0.3, inplace=True),
                                    nn.Linear(in_features=1408,
                                              out_features=3))

In [10]:
def create_effnetb2_model(num_classes:int=3,
                          seed:int=33):

  effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT

  transforms = effnetb2_weights.transforms()

  model = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

  for param in model.parameters():
    param.requires_grad = False

  torch.manual_seed(seed)

  model.classifier = nn.Sequential(nn.Dropout(p=0.3, inplace=True),
                                    nn.Linear(in_features=1408,
                                              out_features=num_classes))

  return model, transforms

In [11]:
effnetb2, effnetb2_transforms = create_effnetb2_model(num_classes=3,
                                                      seed=33)

effnetb2, effnetb2_transforms

(EfficientNet(
   (features): Sequential(
     (0): Conv2dNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): SiLU(inplace=True)
     )
     (1): Sequential(
       (0): MBConv(
         (block): Sequential(
           (0): Conv2dNormActivation(
             (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
             (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2): SiLU(inplace=True)
           )
           (1): SqueezeExcitation(
             (avgpool): AdaptiveAvgPool2d(output_size=1)
             (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
             (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
             (activation): SiLU(inplace=True)
             (scale_activation): Sigmoid()
           )
          

# Creating DataLoaders

In [12]:
from going_modular.going_modular import data_setup
import torchvision

BATCH_SIZE=32

train_dataloader, test_dataloader , class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                test_dir=test_dir,
                                                                                transform=effnetb2_transforms,
                                                                                batch_size=BATCH_SIZE)

In [13]:
train_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f4ea13cfe50>,
 ['pizza', 'steak', 'sushi'])

In [14]:
from going_modular.going_modular import engine

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=effnetb2.parameters(),
                             lr=0.001)

effnetb2_results = engine.train(model=effnetb2,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epochs=3,
             device=device)

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


KeyboardInterrupt: 

In [ ]:
plot_loss_curves(effnetb2_results)

In [ ]:
from going_modular.going_modular import utils

# Save the model
utils.save_model(model=effnetb2,
                 target_dir="models",
                 model_name="09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth")

In [ ]:
from pathlib import Path

pretrained_effnetb2_model_size = Path('models/09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth').stat().st_size / (1024 * 1024)
print(f'Your model size is {pretrained_effnetb2_model_size:.2f} mb')

# effnetb2 stats

In [ ]:
#num of params
effnetb2_total_params = sum(torch.numel(param) for param in effnetb2.parameters())
effnetb2_total_params

In [ ]:
# effnetb2 dic stats
effnetb2_stats = {'test_loss': effnetb2_results['test_loss'][-1],
                  'test_acc': effnetb2_results['test_acc'][-1],
                  'number_of_parameters': effnetb2_total_params,
                  'model_size (MB)': pretrained_effnetb2_model_size}

effnetb2_stats


# Creating ViT model

In [ ]:
vit = torchvision.models.vit_b_16()
vit.heads

In [ ]:
def create_vitb16_model(num_classes:int=3,
                        seed:int=33):
  weights = torchvision.models.ViT_B_16_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.vit_b_16(weights=weights)

  for param in model.parameters():
    param.requires_grad = False

  torch.manual_seed(seed)
  model.heads = nn.Sequential(nn.Linear(in_features=768,
                                         out_features=num_classes))

  return model, transforms

In [ ]:
def create_vit_model(num_classes:int=3,
                     seed:int=42):
    """Creates a ViT-B/16 feature extractor model and transforms.

    Args:
        num_classes (int, optional): number of target classes. Defaults to 3.
        seed (int, optional): random seed value for output layer. Defaults to 42.

    Returns:
        model (torch.nn.Module): ViT-B/16 feature extractor model.
        transforms (torchvision.transforms): ViT-B/16 image transforms.
    """
    # Create ViT_B_16 pretrained weights, transforms and model
    weights = torchvision.models.ViT_B_16_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.vit_b_16(weights=weights)

    # Freeze all layers in model
    for param in model.parameters():
        param.requires_grad = False

    # Change classifier head to suit our needs (this will be trainable)
    torch.manual_seed(seed)
    model.heads = nn.Sequential(nn.Linear(in_features=768, # keep this the same as original model
                                          out_features=num_classes)) # update to reflect target number of classes

    return model, transforms

In [ ]:
# Create ViT model and transforms
vit, vit_transforms = create_vit_model(num_classes=3,
                                       seed=42)


In [ ]:
# Setup ViT DataLoaders
from going_modular.going_modular import data_setup
train_dataloader_vit, test_dataloader_vit, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                       test_dir=test_dir,
                                                                                       transform=vit_transforms,
                                                                                       batch_size=32)

In [ ]:
from going_modular.going_modular import engine

# Setup optimizer
optimizer = torch.optim.Adam(params=vit.parameters(),
                             lr=1e-3)
# Setup loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Train ViT model with seeds set for reproducibility
set_seeds()
vit_results = engine.train(model=vit,
                           train_dataloader=train_dataloader_vit,
                           test_dataloader=test_dataloader_vit,
                           epochs=3,
                           optimizer=optimizer,
                           loss_fn=loss_fn,
                           device=device)

In [ ]:
from helper_functions import plot_loss_curves

plot_loss_curves(vit_results)

In [ ]:
from going_modular.going_modular import utils

# Save the model
utils.save_model(model=vit,
                 target_dir="models",
                 model_name="09_pretrained_vitb16_feature_extractor_pizza_steak_sushi_20_percent.pth")

In [ ]:
from pathlib import Path

pretrained_vit_model_size = Path('models/09_pretrained_vitb16_feature_extractor_pizza_steak_sushi_20_percent.pth').stat().st_size / (1024 * 1024)
print(f'Your model size is {pretrained_vit_model_size:.2f} mb')

In [ ]:
#num of params
vit_total_params = sum(torch.numel(param) for param in vit.parameters())
vit_total_params

In [ ]:
# effnetb2 dic stats
vit_stats = {'test_loss': vit_results['test_loss'][-1],
                  'test_acc': vit_results['test_acc'][-1],
                  'number_of_parameters': vit_total_params,
                  'model_size (MB)': pretrained_vit_model_size}

vit_stats

In [ ]:
test_data_paths = list(Path(test_dir).glob('*/*.jpg'))
test_data_paths

In [ ]:
import pathlib
import torch
import torchvision

from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import List, Dict


def pred_and_store(paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   class_names: List[str],
                   device: str = 'cuda' if torch.cuda.is_available() else 'cpu') ->List[Dict]:

  pred_list = []

  for path in tqdm(paths):
    pred_dict = {}
    pred_dict['image_path'] = path
    class_name = path.parent.stem
    pred_dict['class_name'] = class_name

    start_time = timer()
    img = Image.open(path)

    transformed_image = transform(img).unsqueeze(0).to(device)

    model = model.to(device)
    model.eval()

    with torch.inference_mode():
      pred_logit = model(transformed_image) # perform inference on target sample
      pred_prob = torch.softmax(pred_logit, dim=1) # turn logits into prediction probabilities
      pred_label = torch.argmax(pred_prob, dim=1) # turn prediction probabilities into prediction label
      pred_class = class_names[pred_label.cpu()] # hardcode prediction class to be on CPU

      # 11. Make sure things in the dictionary are on CPU (required for inspecting predictions later on)
      pred_dict["pred_prob"] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
      pred_dict["pred_class"] = pred_class

      end_time = timer()
      pred_dict['time_for_pred'] = round(end_time-start_time, 4)


    pred_dict['correct'] = class_name == pred_class

    pred_list.append(pred_dict)

  return pred_list


In [ ]:
effnetb2_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                          model=effnetb2,
                                          transform=effnetb2_transforms,
                                          class_names=class_names,
                                          device='cpu')

In [ ]:
effnetb2_test_pred_dicts

In [ ]:
import pandas as pd
effnetb2_test_pred_df = pd.DataFrame(effnetb2_test_pred_dicts)
effnetb2_test_pred_df.head()

In [ ]:
# Find the average time per prediction
effnetb2_average_time_per_pred = round(effnetb2_test_pred_df.time_for_pred.mean(), 4)
print(f"EffNetB2 average time per prediction: {effnetb2_average_time_per_pred} seconds")

In [ ]:
vit_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                          model=vit,
                                          transform=vit_transforms,
                                          class_names=class_names,
                                          device='cpu')

In [ ]:
import pandas as pd
vit_test_pred_df = pd.DataFrame(vit_test_pred_dicts)
vit_test_pred_df.head()

In [ ]:
# Find the average time per prediction
vit_average_time_per_pred = round(vit_test_pred_df.time_for_pred.mean(), 4)
print(f"ViT average time per prediction: {vit_average_time_per_pred} seconds")

In [ ]:
# Count the number of correct predictions
vit_test_pred_df.correct.value_counts()

In [ ]:
# Add average prediction time for ViT model on CPU
vit_stats["time_per_pred_cpu"] = vit_average_time_per_pred
vit_stats

In [ ]:
# Add EffNetB2 average prediction time to stats dictionary
effnetb2_stats["time_per_pred_cpu"] = effnetb2_average_time_per_pred
effnetb2_stats

In [ ]:
df = pd.DataFrame([effnetb2_stats, vit_stats])

df['model'] = ['EffNetB2', 'ViT']

df['test_acc'] = round(df['test_acc'] * 100, 2)
df

In [ ]:
pd.DataFrame(data=(df.set_index('model').loc['ViT'] / df.set_index('model').loc['EffNetB2']),
             columns=['ViT to EffNetB2 ratios']).T

In [ ]:
df

In [ ]:
# 1. Create a plot from model comparison DataFrame
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(data=df,
                     x="time_per_pred_cpu",
                     y="test_acc",
                     c=["blue", "orange"], # what colours to use?
                     s="model_size (MB)") # size the dots by the model sizes

# 2. Add titles, labels and customize fontsize for aesthetics
ax.set_title("FoodVision Mini Inference Speed vs Performance", fontsize=18)
ax.set_xlabel("Prediction time per image (seconds)", fontsize=14)
ax.set_ylabel("Test accuracy (%)", fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid('True')

for index, row in df.iterrows():
  ax.annotate(text=row["model"], # note: depending on your version of Matplotlib, you may need to use "s=..." or "text=...", see: https://github.com/faustomorales/keras-ocr/issues/183#issuecomment-977733270
                xy=(row["time_per_pred_cpu"]+0.0006, row["test_acc"]+0.03),
                size=12)

handles, labels = scatter.legend_elements(prop='sizes', alpha=0.5)
model_size_legend = ax.legend(handles,
                              labels,
                              loc='lower right',
                              title='Model size (MB)',
                              fontsize=12)

# Save the figure
plt.savefig("09-foodvision-mini-inference-speed-vs-performance.jpg")

# Show the figure
plt.show()

# Creating a Gradio Demo

In [ ]:
# Import/install Gradio
try:
    import gradio as gr
except:
    !pip -q install gradio
    import gradio as gr

print(f"Gradio version: {gr.__version__}")

#Creating a function to map our inputs and outputs

In [ ]:
effnetb2 = effnetb2.to('cpu')

next(iter(effnetb2.parameters())).device

In [ ]:
from typing import Tuple, Dict

def predict(img) -> Tuple[Dict, float]:

  start_time = timer()

  #transform image
  img = effnetb2_transforms(img).unsqueeze(0)

  #put model in eval mode and make prediction
  effnetb2.eval()
  with torch.inference_mode():
    pred_probs = torch.softmax(effnetb2(img), dim=1)

  #create a pred label and pred probability dict
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  #calc pred time
  end_time = timer()
  pred_time = round(end_time - start_time, 4)
  return pred_labels_and_probs, pred_time

In [ ]:
import random
from PIL import Image

#Get a list of all test image filepath
test_data_paths = list(Path(test_dir).glob('*/*.jpg'))

#randomly select a test image path
random_image_path = random.sample(test_data_paths, k=1)[0]

#open target image
image = Image.open(random_image_path)
print(f'[INFO] Predict on image at path {random_image_path}\n')

#Predict on random image and print out the outputs
pred_dict, pred_time = predict(img=image)
print(pred_dict)
print(pred_time)

# Create a list of examples

In [ ]:
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

#Build a Gradio Interface

In [ ]:
import gradio as gr

title = 'FoodVision Mini 🍕🥩🍣'
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as pizza, steak or sushi."
#article = "Created at [09. PyTorch Model Deployment]"

demo = gr.Interface(inputs=gr.Image(type='pil'),
                    outputs=[gr.Label(num_top_classes=3, label='Predictions'),
                             gr.Number(label='Prediction time (s)')],
                    examples=example_list,
                    fn=predict,
                    title=title,
                    description=description)

demo.launch(debug=False,
            share=True)

# Turning Gradio Demot into a deployable app

In [ ]:
import shutil
from pathlib import Path

#create demo path
foodvision_mini_demo_path = Path('demos/foodvision_mini/')

#remove files that might exist and create a new directory
if foodvision_mini_demo_path.exists():
  shutil.rmtree(foodvision_mini_demo_path)
  foodvision_mini_demo_path.mkdir(parents=True,
                                  exist_ok=True)
else:
  foodvision_mini_demo_path.mkdir(parents=True,
                                  exist_ok=True)

!ls demos/foodvision_mini/

In [ ]:
import shutil
from pathlib import Path

#create an example direct
foodvision_mini_example_path = foodvision_mini_demo_path / 'examples'
foodvision_mini_example_path.mkdir(parents=True,
                                   exist_ok=True)

# 2. Collect three random test dataset image paths
foodvision_mini_examples = [Path('data/pizza_steak_sushi_20_percent/test/sushi/592799.jpg'),
                            Path('data/pizza_steak_sushi_20_percent/test/steak/3622237.jpg'),
                            Path('data/pizza_steak_sushi_20_percent/test/pizza/2582289.jpg')]

# copy the 3 images to the examples dir
for example in foodvision_mini_examples:
  destination = foodvision_mini_example_path / example.name
  print(f'[INFO] Copying {example} to {destination}')
  shutil.copy2(src=example,
               dst=destination)

In [ ]:
import os
example_list = [['examples/' + example] for example in os.listdir(foodvision_mini_example_path)]
example_list

In [ ]:
# Moving the effnetb2 model

import shutil
#source
effnetb2_foodvision_mini_model_path = '/content/models/09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth'

#destination
effnetb2_foodvision_mini_model_destination = foodvision_mini_demo_path / effnetb2_foodvision_mini_model_path.split("/")[-1]

# try to move the model file
try:
    print(f"[INFO] Attempting to move {effnetb2_foodvision_mini_model_path} to {effnetb2_foodvision_mini_model_destination}")

    # Move the model
    shutil.move(src=effnetb2_foodvision_mini_model_path,
                dst=effnetb2_foodvision_mini_model_destination)

    print(f"[INFO] Model move complete.")

# If the model has already been moved, check if it exists
except:
    print(f"[INFO] No model found at {effnetb2_foodvision_mini_model_path}, perhaps its already been moved?")
    print(f"[INFO] Model exists at {effnetb2_foodvision_mini_model_destination}: {effnetb2_foodvision_mini_model_destination.exists()}")

# Turning effnetb2 model into a python script

In [ ]:
%%writefile demos/foodvision_mini/model.py
import torch
import torchvision
from torch import nn

def create_effnetb2_model(num_classes:int=3,
                          seed:int=33):

  effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT

  transforms = effnetb2_weights.transforms()

  model = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

  for param in model.parameters():
    param.requires_grad = False

  torch.manual_seed(seed)

  model.classifier = nn.Sequential(nn.Dropout(p=0.3, inplace=True),
                                    nn.Linear(in_features=1408,
                                              out_features=num_classes))

  return model, transforms

In [ ]:
class_names

In [ ]:
%%writefile demos/foodvision_mini/app.py
# imports and class_names
import gradio as gr
import os
import torch

from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

class_names = ['pizza', 'steak', 'sushi']

# model and transforms
effnetb2, effnetb2_transforms = create_effnetb2_model(
    num_classes = len(class_names)
)


effnetb2.load_state_dict(
    torch.load(
        f'09_pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth',
        map_location=torch.device('cpu')
    )
)

# pred fn
def predict(img) -> Tuple[Dict, float]:

  start_time = timer()

  #transform image
  img = effnetb2_transforms(img).unsqueeze(0)

  #put model in eval mode and make prediction
  effnetb2.eval()
  with torch.inference_mode():
    pred_probs = torch.softmax(effnetb2(img), dim=1)

  #create a pred label and pred probability dict
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  #calc pred time
  end_time = timer()
  pred_time = round(end_time - start_time, 4)
  return pred_labels_and_probs, pred_time

#Gradio app

title = 'FoodVision Mini 🍕🥩🍣'
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as pizza, steak or sushi."

#create example list
example_list = [['examples/' + example] for example in os.listdir('examples')]

demo = gr.Interface(inputs=gr.Image(type='pil'),
                    outputs=[gr.Label(num_top_classes=3, label='Predictions'),
                             gr.Number(label='Prediction time (s)')],
                    examples=example_list,
                    fn=predict,
                    title=title,
                    description=description)

demo.launch(debug=False,
            share=True)

# Creating requirements.txt

In [ ]:
%%writefile demos/foodvision_mini/requirements.txt
torch==2.2.2
torchvision==0.17.2
gr==4.26.0

# Download the files and then upload them

In [ ]:
!cd demos/foodvision_mini && zip -r ../foodvision_mini.zip * -x '*.pyc' '*.ipynb' '*.pycache*' '*.ipynb_checkpoints*'

In [ ]:
# #downloading
# try:
#     from google.colab import files
#     files.download("demos/foodvision_mini.zip")
# except:
#     print("Not running in Google Colab, can't use google.colab.files.download(), please manually download.")

# Creating a model for FooVision Big

In [ ]:
effnetb2_food101, effnetb2_transforms = create_effnetb2_model(num_classes=101)

In [ ]:
effnetb2_transforms

In [ ]:
#Training data
food101_train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.TrivialAugmentWide(),
    effnetb2_transforms])

In [ ]:
food101_train_transforms

In [ ]:
#testing data
effnetb2_train_transforms = effnetb2_transforms

In [ ]:
from torchvision import datasets

#setup data directory
from pathlib import Path
data_dir = Path('data')

#Get_training data
train_data = datasets.Food101(root=data_dir,
                              split='train',
                              transform=food101_train_transforms,
                              download=True)

# #Get testing data
# test_data = datasets.Food101(root=data_dir,
#                              split='test',
#                              transform=effnetb2_transforms,#dont perform data qugumentation
#                              download=True)

In [ ]:
#Get testing data
test_data = datasets.Food101(root=data_dir,
                             split='test',
                             transform=effnetb2_transforms,#dont perform data qugumentation
                             download=True)

In [ ]:
food101_class_names = train_data.classes
food101_class_names[:25]

#Create a subset for faster experimenting

In [ ]:
from torch.utils.data import random_split

def split_datasets(dataset:torchvision.datasets,
                   split_size:float=0.2,
                   seed:int=33):
  #create split lenghts
  length_1 = int(len(dataset) * split_size)
  length_2 = len(dataset) - length_1

  print(f'[INFO] Spliting dataset of length {len(dataset)} into splits of size : {length_1} and {length_2}')

  #Create splits with given random seed
  random_split_1, random_split_2 = torch.utils.data.random_split(dataset,
                                                                 lengths=[length_1, length_2],
                                                                 generator=torch.manual_seed(seed))

  return random_split_1, random_split_2

In [ ]:
train_data_food101_20_percent, _ = split_datasets(dataset=train_data,
                                                  split_size=0.2)

test_data_food101_20_percent, _ = split_datasets(dataset=test_data,
                                                 split_size=0.2)

In [ ]:
BATCH_SIZE = 32
NUM_WORKERS = 2

train_101_dataloader= torch.utils.data.DataLoader(dataset=train_data_food101_20_percent,
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=NUM_WORKERS)

test_101_dataloader= torch.utils.data.DataLoader(dataset=test_data_food101_20_percent,
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=False,
                                                  num_workers=NUM_WORKERS)

# **Training**

In [ ]:
from going_modular.going_modular import engine

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(params=effnetb2_food101.parameters(),
                             lr=0.001)

effnetb2_food101_results = engine.train(model=effnetb2_food101,
                                        train_dataloader=train_101_dataloader,
                                        test_dataloader=test_101_dataloader,
                                        optimizer=optimizer,
                                        loss_fn=loss_fn,
                                        epochs=5,
                                        device=device)

In [ ]:
from helper_functions import plot_loss_curves

plot_loss_curves(effnetb2_food101_results)

In [ ]:
from going_modular.going_modular import utils

# Create a model path
effnetb2_food101_model_path = "09_pretrained_effnetb2_feature_extractor_food101_20_percent.pth"

# Save FoodVision Big model
utils.save_model(model=effnetb2_food101,
                 target_dir="models",
                 model_name=effnetb2_food101_model_path)

In [ ]:
loaded_effnetb2_food101, effnetb2_transforms =create_effnetb2_model(num_classes=101)

#load the saved models state dict
loaded_effnetb2_food101.load_state_dict(torch.load('models/09_pretrained_effnetb2_feature_extractor_food101_20_percent.pth'))

In [ ]:
from pathlib import Path

# Get the model size in bytes then convert to megabytes
pretrained_effnetb2_food101_model_size = Path("models", effnetb2_food101_model_path).stat().st_size // (1024*1024) # division converts bytes to megabytes (roughly)
print(f"Pretrained EffNetB2 feature extractor Food101 model size: {pretrained_effnetb2_food101_model_size} MB")

# 100% of the data

In [ ]:
BATCH_SIZE = 32
NUM_WORKERS = 2

train_101_dataloader_big = torch.utils.data.DataLoader(dataset=train_data,
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=NUM_WORKERS)

test_101_dataloader_big = torch.utils.data.DataLoader(dataset=test_data,
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=False,
                                                  num_workers=NUM_WORKERS)

In [ ]:
len(train_101_dataloader_big)

In [ ]:
from going_modular.going_modular import engine

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(params=effnetb2_food101.parameters(),
                             lr=0.001)

effnetb2_food101_big_results = engine.train(model=effnetb2_food101,
                                        train_dataloader=train_101_dataloader,
                                        test_dataloader=test_101_dataloader_big,
                                        optimizer=optimizer,
                                        loss_fn=loss_fn,
                                        epochs=5,
                                        device=device)

In [ ]:
from helper_functions import plot_loss_curves

plot_loss_curves(effnetb2_food101_big_results)

In [ ]:
from going_modular.going_modular import utils

# Create a model path
effnetb2_food101_big_model_path = "09_pretrained_effnetb2_feature_extractor_food101_big.pth"

# Save FoodVision Big model
utils.save_model(model=effnetb2_food101,
                 target_dir="models",
                 model_name=effnetb2_food101_big_model_path)

In [ ]:
loaded_effnetb2_food101_big, effnetb2_transforms =create_effnetb2_model(num_classes=101)

#load the saved models state dict
loaded_effnetb2_food101_big.load_state_dict(torch.load('models/09_pretrained_effnetb2_feature_extractor_food101_big.pth'))

In [ ]:
from pathlib import Path

# Get the model size in bytes then convert to megabytes
pretrained_effnetb2_food101_big_model_size = Path("models", effnetb2_food101_big_model_path).stat().st_size // (1024*1024) # division converts bytes to megabytes (roughly)
print(f"Pretrained EffNetB2 feature extractor Food101_big model size: {pretrained_effnetb2_food101_big_model_size} MB")